# Create an Advanced Pipeline
This pipeline will perform hyper-parameter tuning with Kubeflow Katib, model training with TensorFlow, model deploying with TensorFlow Serving,  model predicting with a sample web-based UI.

In [ ]:
!pip3 install Pillow kfp --upgrade --user
!pip install -q awscli

In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%%bash

echo "Region: $(curl -s http://169.254.169.254/latest/meta-data/placement/availability-zone | sed 's/\(.*\)[a-z]/\1/')"

In [ ]:
import boto3

#################################
#################################
# REPLACE AWS_REGION= with the current region
#  surround with single quotes
AWS_REGION='us-west-2'
#################################
#################################

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

In [ ]:
import random
import string
from src.mnist.src import katib_launch_args, converter, resource_provider, tfjoblaunch_args_provider
import kfp
from kfp import components
from kfp.components import func_to_container_op
import kfp.dsl as dsl

# Setup S3 Bucket

In [ ]:
mnist_bucket=S3_BUCKET
s3_bucket_path='s3://{}'.format(mnist_bucket)

print(s3_bucket_path)

# Build Kubeflow Pipeline
Note:  This pipeline is hard-coded to `us-west-2`

In [ ]:
namespace='kubeflow'
HASH = ''.join([random.choice(string.ascii_lowercase) for n in range(16)] + [random.choice(string.digits) for n in range(16)])

@dsl.pipeline(
    name="End to end pipeline",
    description="An end to end example including hyperparameter tuning, train and inference."
)
def mnist_pipeline(
        name="mnist-{{workflow.uid}}",
        namespace=namespace,
        step="1000",
        s3bucketexportpath="",
        ttlSecondsAfterFinished=-1,
        tfjobTimeoutMinutes=60,
        deleteAfterDone=False):

    # step 1: create a Katib experiment to tune hyperparameters
    objectiveConfig, algorithmConfig, parameters, trialTemplate, metricsCollectorSpec =  \
        katib_launch_args.argugments_provide(objective_type="minimize",
                                             objective_goal=0.001,
                                             objective_metrics="loss",
                                             algorithm="random",
                                             parameters_lr_min="0.01",
                                             parameters_lr_max="0.03",
                                             parameters_batchsize=["16", "32", "64"],
                                             tf_train_steps="200",
                                             image="chuckshow/mnist-tf-pipeline:latest",
                                             worker_num=3)
                          

    katib_experiment_launcher_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml')
    op1 = katib_experiment_launcher_op(
            experiment_name=name,
            experiment_namespace=namespace,
            parallel_trial_count=3,
            max_trial_count=12,
            objective=str(objectiveConfig),
            algorithm=str(algorithmConfig),
            trial_template=str(trialTemplate),
            parameters=str(parameters),
            metrics_collector=str(metricsCollectorSpec),
            delete_finished_experiment=False)
    
    # step 1.5: convert Katib best parameteres into string
    convert_op = func_to_container_op(converter.convert_mnist_experiment_result)
    op2 = convert_op(op1.output)
    
    # step2: create a TFJob Launcher to train your model with best hyperparameter tuned by Katib
    tfjob_launcher_op = components.load_component_from_file("./src/mnist/launcher/component.yaml")
    
    chief, worker = tfjoblaunch_args_provider.tfjoblauncher_args(step=step, 
                                                                 s3bucketexportpath=s3bucketexportpath, 
                                                                 args=op2.output,
                                                                 aws_region=AWS_REGION)
    
    train = tfjob_launcher_op(
        name=name,
        namespace=namespace,
        ttl_seconds_after_finished=ttlSecondsAfterFinished,
        worker_spec=worker,
        chief_spec=chief,
        tfjob_timeout_minutes=tfjobTimeoutMinutes,
        delete_finished_tfjob=deleteAfterDone,
    )
    
    # step 3: model inferencese by Tensorflow Serving    
    servingdeploy_name = 'mnist-model' + HASH
    
    deploy = resource_provider.tfservingdeploy_resource(namespace=namespace,
                                                       s3bucketexportpath=s3bucketexportpath,
                                                       servingdeploy_name=servingdeploy_name,
                                                       aws_region=AWS_REGION)
    
    deployment = dsl.ResourceOp(
        name="deploy",
        k8s_resource=deploy,
    ).after(train)
    
    servingsvc_name = 'mnist-service'
    serviceresource = resource_provider.tfservingsvc_resource(namespace=namespace,
                                                             servingdeploy_name=servingdeploy_name,
                                                             servingsvc_name=servingsvc_name)
    
    service = dsl.ResourceOp(
        name="service",
        k8s_resource=serviceresource
    ).after(deployment)
    

    # step 4: mnist ui deploy
    ui_name = 'mnist-ui' + HASH
    uideployresource = resource_provider.uideploy_resource(namespace=namespace,
                                                          ui_name=ui_name)
    
    uideploy = dsl.ResourceOp(
        name="uideploy",
        k8s_resource=uideployresource
    ).after(train)
   
    uiserviceresource = resource_provider.uisvc_resource(namespace=namespace,
                                                        ui_name=ui_name)
    
    uiservice = dsl.ResourceOp(
        name="uiservice",
        k8s_resource=uiserviceresource
    ).after(uideploy)
    
    uivirtualserviceresource = resource_provider.uivirtualsvc_resource(namespace=namespace,
                                                                      ui_name=ui_name)
    
    uivirtualservice = dsl.ResourceOp(
        name="uivirtualservice",
        k8s_resource=uivirtualserviceresource
    ).after(uiservice)
    

### Submit the pipeline

In [ ]:
pipeline = kfp.Client().create_run_from_pipeline_func(mnist_pipeline, arguments={"s3bucketexportpath":'{}/export'.format(s3_bucket_path)})

# Navigate to the Experiment and Run links ^^ Above 

# _YOU MUST WAIT FOR THE PIPELINE TO COPMLETE BEFORE CONTINUING_
Otherwise, you will see errors below!!

### Invoke serving API via Python client

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import os
import subprocess
import argparse

import random
import json
import requests

# Note:  Port 8500 is HTTP/REST API.
endpoint = "http://mnist-service.{}.svc.cluster.local:8500/v1/models/mnist:predict".format(namespace)

# Prepare test dataset
fashion_mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['0','1','2','3','4','5','6','7','8','9']

# Random generate one image
rando = random.randint(0,len(test_images)-1)
data = json.dumps({"signature_name": "serving_default", "instances": test_images[rando:rando+1].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

# _IF YOU SEE A CONNECTION ERROR BELOW, YOU NEED TO WAIT FOR THE PIPELINE TO DEPLOY_

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post(endpoint, data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print(predictions)

title = 'The model thought this was a class {}, and it was actually a class {}'.format(
test_labels[rando], predictions[0]['classes'])
print('\n')
print(title)

# _IF YOU SEE A CONNECTION ^^ ERROR ^^ ABOVE, YOU NEED TO WAIT FOR THE PIPELINE TO DEPLOY_

### Invoke Serving API through a Web UI
Navigate to the following URL to make predictions using the sample application.  

_Replace the **[istio-endpoint]** below with the same **hostname** of the browser url for this current Kubeflow notebook.  Just the **hostname**._

```
http://<istio-endpoint>/mnist/kubeflow/ui/ <== REQUIRES TRAILING SLASH
```

_Replace the **[istio-endpoint]** above with the same **hostname** of the browser url for this current Kubeflow notebook.  Just the **hostname**._  

### (Optional) If You Re-Run This Notebook, You Will First Need to Delete Some Resources
Note:  If you re-run this notebook, you will first need to delete the existing `mnist-service` using the following command **in your SageMaker notebook** (not this Kubeflow notebook - you won't have access.):

In [ ]:
# mnistuisvc_name = "mnist-ui" + HASH
# mnistmodel_name= "mnist-model" + HASH

# print('This is not required, but if you want to clean up the resources, you would execute the following commands in the SageMaker notebook - not in this Kubeflow notebook')
# print('')
# print('kubectl delete svc -n kubeflow mnist-service')
# print('kubectl delete svc -n kubeflow {}'.format(mnistuisvc_name))
# print('kubectl delete deploy -n kubeflow {}'.format(mnistmodel_name))
# print('kubectl delete deploy -n kubeflow {}'.format(mnistuisvc_name))
# print('kubectl delete virtualservice -n kubeflow {}'.format(mnistuisvc_name))